## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [10]:
import mlflow
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/home/harsha/.config/gcloud/application_default_credentials.json' # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "35.233.97.115" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://35.233.97.115:5000'


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='gs://mlflow-bucket-mlops/0', creation_time=1685747476990, experiment_id='0', last_update_time=1685747476990, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 'gs://mlflow-bucket-mlops/1/825258aedc9c4558b0319c22b217724d/artifacts'


In [12]:
mlflow.search_experiments()

[<Experiment: artifact_location='gs://mlflow-bucket-mlops/1', creation_time=1685748114597, experiment_id='1', last_update_time=1685748114597, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='gs://mlflow-bucket-mlops/0', creation_time=1685747476990, experiment_id='0', last_update_time=1685747476990, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [13]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [37]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1685749931174, description='', last_updated_timestamp=1685749931341, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685749931341, current_stage='None', description='', last_updated_timestamp=1685749931341, name='gcp-setup', run_id='825258aedc9c4558b0319c22b217724d', run_link='', source='gs://mlflow-bucket-mlops/1/825258aedc9c4558b0319c22b217724d/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='gcp-setup', tags={}>]

In [36]:
run_id = client.list_run_infos(experiment_ids='1')[0]
print(run_id)
# mlflow.register_model(
#     model_uri=f"runs:/{run_id}/models",
#     name='iris-classifier'
# )

AttributeError: 'MlflowClient' object has no attribute 'list_run_infos'